#**Deep Neural Network for Horses or Humans Classification**
# TensorFlow practise on image classification. The dataset that I will use is the horses_or_humans dataset that is part of the TensorFlow datasets.


## Import the relevant libraries

In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [ ]:
image_dataset, image_info = tfds.load(name='horses_or_humans', with_info=True ,as_supervised=True)
image_train, image_test = image_dataset['train'], image_dataset['test']

num_validation_samples = 0.1 * image_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64) ##convert variable into int

num_test_samples = image_info.splits["test"].num_examples
num_test_samples - tf.cast(num_test_samples, tf.int64)

#scale the images
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

scaled_train_and_validation_data = image_train.map(scale)

test_data = image_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 128

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# **Model**

#### **Outline the model**

In [ ]:
input_size = 90000
output_size = 10
hidden_layer_size = 1000

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(300,300,3)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax'),
                            ])

#### **Choose the optimizer and the loss function**

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# **Training**

In [ ]:
NUM_EPOCHS = 10

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_data, epochs=NUM_EPOCHS, callbacks=[early_stopping], validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/10
10/10 - 23s - loss: 348.2516 - accuracy: 0.4530 - val_loss: 145.4349 - val_accuracy: 0.4902
Epoch 2/10
10/10 - 23s - loss: 121.0512 - accuracy: 0.5384 - val_loss: 111.5363 - val_accuracy: 0.4902
Epoch 3/10
10/10 - 23s - loss: 61.9592 - accuracy: 0.5232 - val_loss: 46.4940 - val_accuracy: 0.4902
Epoch 4/10
10/10 - 23s - loss: 18.7811 - accuracy: 0.5503 - val_loss: 9.4146 - val_accuracy: 0.5490
Epoch 5/10
10/10 - 23s - loss: 3.7731 - accuracy: 0.6530 - val_loss: 6.0587 - val_accuracy: 0.5294
Epoch 6/10
10/10 - 23s - loss: 2.9868 - accuracy: 0.6800 - val_loss: 1.5683 - val_accuracy: 0.7157
Epoch 7/10
10/10 - 23s - loss: 1.1189 - accuracy: 0.7805 - val_loss: 0.5291 - val_accuracy: 0.7941
Epoch 8/10
10/10 - 23s - loss: 0.6746 - accuracy: 0.8141 - val_loss: 0.4927 - val_accuracy: 0.8137
Epoch 9/10
10/10 - 23s - loss: 0.3903 - accuracy: 0.8573 - val_loss: 0.3007 - val_accuracy: 0.8431
Epoch 10/10
10/10 - 23s - loss: 0.2681 - accuracy: 0.8951 - val_loss: 0.2365 - val_accuracy: 0.902

## **Testing**

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 18ms/step - loss: 0.4364 - accuracy: 0.8945


In [ ]:
print('Test loss: {0: .2f}. Test accuracy: {1: .2f}% '.format(test_loss, test_accuracy*100.))

Test loss:  0.44. Test accuracy:  89.45% 


# The accuracy on the test dataset is a little less than the accuracy on the training dataset. The small difference between the test and train accuracy represents 'overfitting'. It essentially memorises the details in the training dataset to a point where the model fails miserably in the test dataset.


